[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# DIY Covid-19 Dashboard

This is a template for your DIY Covid Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the EECS JupyterHub as well as in the binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [48]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [49]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

You should include "canned" data in ```.json``` files along with your dashboard. When the dashboard starts, it should load that data (the code below will be hidden when the dashboard is rendered by Voila).

In [50]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
jsondata={}

### pcr_comp json load ###

with open("pcr_comp.json", "rt") as INFILE:
    pcr_data = json.load(INFILE)

### vax_occ json load ###

with open("sick_vax.json", "rt") as INFILE:
    sick_vax_data = json.load(INFILE)

### cas_nation json load ###

with open("cas_nation_data.json", "rt") as INFILE:
    cas_nat_data = json.load(INFILE)


### death_gender_age load ###

with open("death_data.json", "rt") as INFILE:
    death_data = json.load(INFILE)

with open("sex_data.json", "rt") as INFILE:
    sex_data = json.load(INFILE)  

## Wrangle the data

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and generate a dataframe with some random data

In [51]:


def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    df=pd.DataFrame(index=range(0,100), columns=['One', 'Two'])
    # we have no real data to wrangle, so we just generate two random walks.
    one=two=0.0
    for i in range(0,100):
        df.loc[i,'One']=one
        df.loc[i,'Two']=two
        one+=np.random.randn()
        two+=2*np.random.randn()
    return df

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
df=wrangle_data(jsondata) # df is the dataframe for plotting

### Pre-requisite Functions ###

#function to get the panda of a datestring

def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")


### Wrangling the Data via Functions for each Graph ###

### pcr_comp Data Wrangling ###

def pcr_comp_wrangle(rawdata):
    pcr_comp_data = rawdata['data']
    
    pcr_dates = [dic["date"] for dic in pcr_comp_data] # getting the dates
    pcr_dates.sort()    # sorting the dates

     
    pcr_startdate = parse_date(pcr_dates[0])    # getting the startdate
    pcr_enddate = parse_date(pcr_dates[-1])     # getting the endddate

    pcr_index = pd.date_range(pcr_startdate, pcr_enddate, freq='D')     # getting the index for the dataframe


    pcr_comp_df = pd.DataFrame(index=pcr_index, columns=["newPCR", "plannedPCR"])   # creating the dataframe

    # filling the the dataframe

    for entry in pcr_comp_data:

        date = parse_date(entry["date"])

        for column in ["newPCR", "plannedPCR"]:     # learning: make sure you have the same column name for the data frame and the list / json file

         if pd.isna(pcr_comp_df.loc[date, column]):

            value = float(entry[column]) if entry[column] !=None else 0.0

            pcr_comp_df.loc[date, column] = value

    pcr_comp_df.fillna(0.0, inplace=True)   
    return pcr_comp_df


pcr_comp_df = pcr_comp_wrangle(pcr_data) # mangling the data initally when loading 


###


### vax_occ Data Wrangling ###

def vax_occ_wrangle(rawdata):

    sick_vax_data = rawdata['data']
    #sick_vax_datalist = sick_vax_data["data"]   # getting the data list


    sick_vax_dates = [dictionary["date"] for dictionary in sick_vax_data ]       # getting start and end dates
    sick_vax_dates.sort()

    sick_vax_startdate = parse_date(sick_vax_dates[0])  
    sick_vax_enddate = parse_date(sick_vax_dates[-1])

    sick_vax_index = pd.date_range(sick_vax_startdate, sick_vax_enddate, freq='D')      # creating the index based on dates

    sick_vax_df = pd.DataFrame(index=sick_vax_index, columns=["cumAdmin", "occMVBeds", "cumVax1", "cumVax2", "cumVax3"])    # creating the data frame

    #   fillinf the data frame

    for entry in sick_vax_data:

     date = parse_date(entry["date"])

     for column in ["cumAdmin", "occMVBeds", "cumVax1", "cumVax2", "cumVax3"]: 

            if pd.isna(sick_vax_df.loc[date, column]):

              value = float(entry[column]) if entry[column] !=None else 0.0

              sick_vax_df.loc[date, column] = value

    sick_vax_df.fillna(0.0, inplace=True)
    return sick_vax_df

sick_vax_df = vax_occ_wrangle(sick_vax_data)    # mangling the data initially when loading

### 



### cas_nation Data Wrangling ###

def cas_nation_wrangle(rawdata):

    new_case_nat_data = []                          # here we are getting a list of api data structures. We are iterating over them them to get the data list and in paralell combining that data into one variable
    for data in rawdata:
        new_case_nat_data += data['data']

    cas_nat_data = new_case_nat_data 

    #cas_nat_data = rawdata
    cas_nat_dates_raw=[dictionary['date'] for dictionary in cas_nat_data ]
    cas_nat_dates_raw.sort()


    cas_nat_dates_dic = {date for date in cas_nat_dates_raw} # removing duplicates by transforming into a dictionary and back into a sorted list
    cas_nat_dates = list(cas_nat_dates_dic)
    cas_nat_dates.sort()    # sorting the dates (not required but I wanted the data clean)

    cas_nat_startdate=parse_date(cas_nat_dates[0])  # getting the start and end dates
    cas_nat_enddate=parse_date(cas_nat_dates[-1])

    #cas_nat_index_m=pd.date_range(cas_nat_startdate, cas_nat_enddate, freq='M') # creating the index for the dataframe with monthly frequency (do i need this? currently not returned)
    cas_nat_index_d=pd.date_range(cas_nat_startdate, cas_nat_enddate, freq='D') # creating the index for the dataframe with daily frequency

    cas_nat_df_d = pd.DataFrame(index=cas_nat_index_d, columns = ("England", "Scotland", "Wales", "Northern Ireland"))
    print(cas_nat_df_d) # building a dataframe based on daily frequency

    #cas_nat_df_m = pd.DataFrame(index=cas_nat_index_m, columns = ("England", "Scotland", "Wales", "Northern Ireland"))
    #print(cas_nat_df_m) # building a dataframe based on monthly frequency

    # filling the dataframes by iterating over the data and having conditional löogic based on the Area Name and Date columns

    for dictionary in cas_nat_data:
        date=parse_date(dictionary['date'])

        if dictionary['areaName'] == "England":

            value= float(dictionary['newCases']) if dictionary['newCases']!=None else 0.0

            cas_nat_df_d.loc[date, "England"]= value

        if dictionary['areaName'] == "Scotland":

            value= float(dictionary['newCases']) if dictionary['newCases']!=None else 0.0

            cas_nat_df_d.loc[date, "Scotland"]= value


        if dictionary['areaName'] == "Wales":

            value= float(dictionary['newCases']) if dictionary['newCases']!=None else 0.0

            cas_nat_df_d.loc[date, "Wales"]= value

        if dictionary['areaName'] == "Northern Ireland":

            value= float(dictionary['newCases']) if dictionary['newCases']!=None else 0.0

            cas_nat_df_d.loc[date, "Northern Ireland"]= value

        cas_nat_df_d.fillna(0.0, inplace=True)  # filling the dataframe with 0.0 for missing values

    return cas_nat_df_d

cas_nat_df_d = cas_nation_wrangle(cas_nat_data) # mangling the data initially when loading

###




### death_gender_age load ###

def death_gender_wrangle(rawdata_death, rawdata_sex):

    death_data = rawdata_death['data']
    sex_data = rawdata_sex['data']


    # getting the timeseries


    death_dates_raw =[dictionary['date'] for dictionary in death_data ]

    death_dates_dic = {date for date in death_dates_raw} # removing duplicates by transforming into a dictionary and back into a sorted list 
    death_dates = list(death_dates_dic)
    death_dates.sort()

    # gettingn start and end dates

    death_startdate=parse_date(death_dates[0])
    death_enddate=parse_date(death_dates[-1])

    death_index_d=pd.date_range(death_startdate, death_enddate, freq='D') # creating the index for the dataframe with daily frequency

    # getting female / male cases and the age ranges / is this needed ???

    female_cases = [dictionary['femaleCases'] for dictionary in sex_data]
    male_cases = [dictionary['maleCases'] for dictionary in sex_data]

    age_bands =[]
    for item in male_cases:
        age_bands += [dictionary['age'] for dictionary in item]

    age_bands = list(set(age_bands)) # removing duplicates and sorting (how did sorting here actually happen? Im not sure. But it sorted for all but one age band so Im doing it fully correct below)
    age_bands.sort()


    def min_age(agerange):
        agerange=agerange.replace('+','') # remove the + from 90+
        start=agerange.split('_')[0]
        return int(start)

    age_bands.sort(key=min_age)


    # building the dataframe for deaths

    death_df = pd.DataFrame(index=death_index_d, columns = ("DailyDeaths", "F_0_to_4", "F_5_to_9", "F_10_to_14", "F_15_to_19", "F_20_to_24", "F_25_to_29", "F_30_to_34", "F_35_to_39", "F_40_to_44", "F_45_to_49", "F_50_to_54", "F_55_to_59", "F_60_to_64", "F_65_to_69", "F_70_to_74", "F_75_to_79", "F_80_to_84", "F_85_to_89", "F_90_to_94", "F_90+", "M_0_to_4", "M_5_to_9", "M_10_to_14", "M_15_to_19", "M_20_to_24", "M_25_to_29", "M_30_to_34", "M_35_to_39", "M_40_to_44", "M_45_to_49", "M_50_to_54", "M_55_to_59", "M_60_to_64", "M_65_to_69", "M_70_to_74", "M_75_to_79", "M_80_to_84", "M_85_to_89", "M_90_to_94", "M_90+"))

    # filling the data frame

    ## filling in the gender / case / age_range data (yes this is a bit redundant – will clean up if I have time. Logic is needed though)


    for dictionary in sex_data:
        date=parse_date(dictionary['date'])
  
        for item in dictionary['femaleCases']:
           
            if item['age'] == '0_to_4':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "F_0_to_4"]= value
            if item['age'] == '5_to_9':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "F_5_to_9"]= value
            if item['age'] == '10_to_14':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "F_10_to_14"]= value
            if item['age'] == '15_to_19':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "F_15_to_19"]= value
            if item['age'] == '20_to_24':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "F_20_to_24"]= value
            if item['age'] == '25_to_29':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "F_25_to_29"]= value
            if item['age'] == '30_to_34':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "F_30_to_34"]= value
            if item['age'] == '35_to_39':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "F_35_to_39"]= value
            if item['age'] == '40_to_44':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "F_40_to_44"]= value
            if item['age'] == '45_to_49':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "F_45_to_49"]= value
            if item['age'] == '50_to_54':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "F_50_to_54"]= value
            if item['age'] == '55_to_59':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "F_55_to_59"]= value
            if item['age'] == '60_to_64':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "F_60_to_64"]= value
            if item['age'] == '65_to_69':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "F_65_to_69"]= value
            if item['age'] == '70_to_74':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "F_70_to_74"]= value
            if item['age'] == '80_to_84':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "F_80_to_84"]= value
            if item['age'] == '85_to_89':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "F_85_to_89"]= value
            if item['age'] == '90_to_94':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "F_90_to_94"]= value
            if item['age'] == '90+':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "F_90+"]= value
        for item in dictionary['maleCases']:
            if item['age'] == '0_to_4':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "M_0_to_4"]= value
            if item['age'] == '5_to_9':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "M_5_to_9"]= value
            if item['age'] == '10_to_14':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "M_10_to_14"]= value
            if item['age'] == '15_to_19':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "M_15_to_19"]= value
            if item['age'] == '20_to_24':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "M_20_to_24"]= value
            if item['age'] == '25_to_29':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "M_25_to_29"]= value
            if item['age'] == '30_to_34':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "M_30_to_34"]= value
            if item['age'] == '35_to_39':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "M_35_to_39"]= value
            if item['age'] == '40_to_44':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "M_40_to_44"]= value
            if item['age'] == '45_to_49':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "M_45_to_49"]= value
            if item['age'] == '50_to_54':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "M_50_to_54"]= value
            if item['age'] == '55_to_59':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "M_55_to_59"]= value
            if item['age'] == '60_to_64':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "M_60_to_64"]= value
            if item['age'] == '65_to_69':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "M_65_to_69"]= value
            if item['age'] == '70_to_74':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "M_70_to_74"]= value
            if item['age'] == '80_to_84':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "M_80_to_84"]= value
            if item['age'] == '85_to_89':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "M_85_to_89"]= value
            if item['age'] == '90_to_94':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "M_90_to_94"]= value
            if item['age'] == '90+':
                value= float(item['value']) if item['value'] !=None else 0.0
                death_df.loc[date, "M_90+"]= value

    ## filling in the daily deaths data into the same datframe 

    for entry in death_data:    # could probably use this method for the gender data to save lines of code... try if I have time

        date = parse_date(entry["date"])

        if pd.isna(death_df.loc[date, "DailyDeaths"]):

            value = float(entry["deaths"]) if entry["deaths"] !=None else 0.0

            death_df.loc[date, "DailyDeaths"] = value


    death_df.fillna(0.0, inplace=True) # fill in the rest of the NaNs with 0.0

    return death_df

death_df = death_gender_wrangle(death_data, sex_data)




           England Scotland Wales Northern Ireland
2020-01-31     NaN      NaN   NaN              NaN
2020-02-01     NaN      NaN   NaN              NaN
2020-02-02     NaN      NaN   NaN              NaN
2020-02-03     NaN      NaN   NaN              NaN
2020-02-04     NaN      NaN   NaN              NaN
...            ...      ...   ...              ...
2022-11-20     NaN      NaN   NaN              NaN
2022-11-21     NaN      NaN   NaN              NaN
2022-11-22     NaN      NaN   NaN              NaN
2022-11-23     NaN      NaN   NaN              NaN
2022-11-24     NaN      NaN   NaN              NaN

[1029 rows x 4 columns]


## Download current data

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting;
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

Clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

In [52]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """

    ### calling the pcr_comp data ###

    filters_pcr_comp = ['areaType=overview']    # creating filter 
    structure_pcr_comp = {"date": "date", "newPCR": "newPCRTestsByPublishDate", "plannedPCR": "plannedPCRCapacityByPublishDate",}   # creating structure

    api_pcr_comp = Cov19API(filters=filters_pcr_comp, structure=structure_pcr_comp)     # calling the API with the filters & structures
    pcr_comp_data_new = api_pcr_comp.get_json()     # getting the json data into a variable

    ### calling the vax_occ data ###

    sick_vax_filters = ['areaType=overview'] # note each metric-value pair is inside one string
    sick_vax_structure = {
        "date": "date",
        "cumAdmin":"cumAdmissions",
        "occMVBeds": "covidOccupiedMVBeds",
        "cumVax1": "cumPeopleVaccinatedFirstDoseByPublishDate",
        "cumVax2": "cumPeopleVaccinatedSecondDoseByPublishDate",
        "cumVax3": "cumPeopleVaccinatedThirdInjectionByPublishDate",
    }

    sick_vax_api = Cov19API(filters=sick_vax_filters, structure=sick_vax_structure)
    sick_vax_data_new = sick_vax_api.get_json()

    ### calling the cas nation data ###

    # filters for each nation

    cas_nation_filters_eng = [
        'areaType=Nation', "areaName=England" # 
    ]

    cas_nation_filters_wales = [
        'areaType=Nation', "areaName=Wales" # 
    ]

    cas_nation_filters_scot = [
        'areaType=Nation', "areaName=Scotland" # 
    ]

    cas_nation_filters_northi = [
        'areaType=Nation', "areaName=Northern Ireland" # 
    ]

    # structure for the data

    cas_nation_structure = {
        "date": "date",
        "areaName": "areaName",
        "newCases": "newCasesByPublishDate"
    }

    # including additonal logic here to combine data and make future function calls easier

    cas_nation_api_eng = Cov19API(filters=cas_nation_filters_eng, structure=cas_nation_structure)
    cas_nation_data_eng = cas_nation_api_eng.get_json()

    cas_nation_api_scot = Cov19API(filters=cas_nation_filters_scot, structure=cas_nation_structure)
    cas_nation_data_scot = cas_nation_api_scot.get_json()

    cas_nation_api_wales = Cov19API(filters=cas_nation_filters_wales, structure=cas_nation_structure)
    cas_nation_data_wales = cas_nation_api_wales.get_json()

    cas_nation_api_northi = Cov19API(filters=cas_nation_filters_northi, structure=cas_nation_structure)
    cas_nation_data_northi = cas_nation_api_northi.get_json()

    cas_nat_data_new = [cas_nation_data_eng, cas_nation_data_scot, cas_nation_data_wales, cas_nation_data_northi] # combining the data

    ### calling the death gender data ###

    # defining the filters

    death_filters = [
        'areaType=Overview'
    ]

    sex_filter = [
        'areaType=nation',
        'areaName=England', # remember to highlight this
    ]

    # defining the structure

    death_structure = {
        "date": "date",
        "deaths": "newDailyNsoDeathsByDeathDate",
    }

    sex_structure = {
        "date": "date",
        "femaleCases": "femaleCases",
        "maleCases": "maleCases",
    }


    death_api = Cov19API(filters=death_filters, structure=death_structure)  # calling the API for the deaths data
    death_data_new = death_api.get_json()

    sex_api = Cov19API(filters=sex_filter, structure=sex_structure)     # calling the APi for the gender + agebands data
    sex_data_new=sex_api.get_json()

    return {'pcrcompdata' : pcr_comp_data_new, 'vaxoccdata' : sick_vax_data_new, 'casnatdata' : cas_nat_data_new, 'deathdata' : death_data_new, 'genderdata' : sex_data_new} # return data read from the API as a dictionary to make it easy to call the right data later on

In [53]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.

    try:
        apidata=access_api()
    except ConnectionError as ConnectionErr:
        print("We could not connect wit the publicly hosted database because the internet connection broke off. Your last refreshed data is still available and has not been overwritten")

    # wrangle the data and overwrite the dataframe for plotting

    # for pcr_comp
    global pcr_comp_df
    pcr_comp_df=pcr_comp_wrangle(apidata['pcrcompdata'])

    # for vaxx occ

    global sick_vax_df
    sick_vax_df = vax_occ_wrangle(apidata['vaxoccdata'])

    global cas_nat_df_d
    cas_nat_df_d = cas_nation_wrangle(apidata['casnatdata'])

    global death_df
    death_df = death_gender_wrangle(apidata['deathdata'], apidata['genderdata'])



    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Refresh Data', # you may want to change this...
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click ro refresh data",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='download'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(button_style='info', description='Refresh Data', icon='download', style=ButtonStyle(), tooltip='Click r…

## Graphs and Analysis

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

Header pcr comp

In [54]:
### having seperate code boxed to plot charts individually? and provide descriptions? probably better



### pcr_comp plotting ###
pcr_cols=wdg.SelectMultiple(
    options=['newPCR', 'plannedPCR'], # could add "delta" later
    value=['newPCR', 'plannedPCR'], # initial value
    rows=2, # rows of the selection box
    description='Categories',
    disabled=False
)


def pcr_comp_graph(graphcolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        pcr_comp_df.plot(kind='line', y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
        plt.show() # important - graphs won't update properly if this is missing
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    
# keep calling age_graph(graphcolumns=value_of_agecols); capture output in widget output    


####

graph_pcr_comp=wdg.interactive_output(pcr_comp_graph, {'graphcolumns': pcr_cols})

display(pcr_cols, graph_pcr_comp)
    


SelectMultiple(description='Categories', index=(0, 1), options=('newPCR', 'plannedPCR'), rows=2, value=('newPC…

Output()

Header Vax Occ

In [55]:
## vax_occ plotting ###

vaxcols=wdg.SelectMultiple(
    options=['cumVax1', 'cumVax2', 'cumVax3'], 
    value=['cumVax1', 'cumVax2', "cumVax3"],
    rows=3, 
    description='Vaccines',
    disabled=False
)

occcols=wdg.SelectMultiple(     # turn this into radio to make clear multi select doesn work? Can multi select work?
    options=['occMVBeds', "cumAdmin"], 
    value=['occMVBeds', "cumAdmin" ], 
    rows=2, 
    description='Sick Metrics',
    disabled=False
)


vax_occ_scale=wdg.RadioButtons(
    options=['linear', 'log'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

controls=wdg.HBox([vaxcols, occcols, vax_occ_scale])

def vax_occ_graph(graphcolumns, ycolumns, gscale):   # check here which values we should name / does it matter?
    
    ncols=len(graphcolumns)

    if gscale=='linear':    # controlling for the liner/log scale
        logscale=False
    else:
        logscale=True

    if occcols.value == ("occMVBeds",):   # why does this work only this way while got gscale, etc. the string works? !!!!
        yaxis_state = "occMVBeds"
        yaxis_desc = "Cummulative Admissions"
    else:                                           # controlling for the secondary y axis
        yaxis_state = "cumAdmin"
        yaxis_desc = "MV Bed Occupancy"


    if ncols>0:
        print(yaxis_state)
        sick_vax_df.plot( y=list(graphcolumns), logy=logscale, use_index=True) 
        ax = sick_vax_df[yaxis_state].plot(secondary_y=True, logy=logscale, color='k')  # plotting a secondary y axis to plot data at different dimensions
        ax.set_ylabel(yaxis_desc)
        plt.show() 
    
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    
# keep calling age_graph(graphcolumns=value_of_agecols); capture output in widget output    
graph_vax_occ=wdg.interactive_output(vax_occ_graph, {'graphcolumns': vaxcols, "ycolumns" : occcols, "gscale":vax_occ_scale})
#output2=wdg.interactive_output(age_graph, {'graphcolumns': occcols})

display(controls, graph_vax_occ) 

Output()

Cas Nat Header

In [56]:

natcols=wdg.SelectMultiple(
    options=['England', 'Scotland', 'Wales', 'Northern Ireland'], # options available
    value=['England', 'Scotland', 'Wales', 'Northern Ireland'], # initial value
    rows=4, # rows of the selection box
    description='Nations',
    disabled=False
)

timescale=wdg.RadioButtons(
    options=['Daily', 'Monthly'],
    description='Scale:',
    disabled=False
)

timeslider=wdg.IntSlider(
    value=3,
    min=1,
    max=12,
    step=1,
    description='Months:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

timeslider2=wdg.IntSlider(
    value=1,
    min=1,
    max=31,
    step=1,
    description='Days:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

controls=wdg.HBox([natcols, timescale])
sliderbox=wdg.HBox([timeslider, timeslider2])



def cas_nat_graph(nationcolumns, tscale, timevalue, timevalue2):

    # our callback function.
    ncols=len(nationcolumns)
    
    m_value = str(timeslider.value) + "m"
    d_value = str(timeslider2.value) + "d"

    if ncols>0:

        if  tscale=='Daily':
            cas_nat_df_dd = cas_nat_df_d.resample(d_value).sum() # had to use the dd naming to have a new variable -> could be cleaner
            cas_nat_df_dd.plot( y=list(nationcolumns))
            plt.show()


        

        else: 
            cas_nat_df_m = cas_nat_df_d.resample(m_value).sum()
            cas_nat_df_m.plot( y=list(nationcolumns), kind='bar', use_index=True)
            plt.show()
            slider_dis = False
            print(slider_dis)
            return slider_dis

    else:
        
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    
  
graph_cas_nat=wdg.interactive_output(cas_nat_graph, {'nationcolumns': natcols, "tscale": timescale, "timevalue": timeslider, "timevalue2":timeslider2}) # clean up slider names + order





display(controls, graph_cas_nat, sliderbox)

Output()

Death Gender Header

In [57]:


female_age_cols=wdg.SelectMultiple(
    options=["F_0_to_4", "F_5_to_9", "F_10_to_14", "F_15_to_19", "F_20_to_24", "F_25_to_29", "F_30_to_34", "F_35_to_39", "F_40_to_44", "F_45_to_49", "F_50_to_54", "F_55_to_59", "F_60_to_64", "F_65_to_69", "F_70_to_74", "F_75_to_79", "F_80_to_84", "F_85_to_89", "F_90_to_94", "F_90+"],
    value=["F_0_to_4", "F_5_to_9", "F_10_to_14", "F_15_to_19", "F_20_to_24", "F_25_to_29", "F_30_to_34", "F_35_to_39", "F_40_to_44", "F_45_to_49", "F_50_to_54", "F_55_to_59", "F_60_to_64", "F_65_to_69", "F_70_to_74", "F_75_to_79", "F_80_to_84", "F_85_to_89", "F_90_to_94", "F_90+"],
    rows= 20, # rows of the selection box
    description='Female Age Bands',
    disabled=False
)

male_age_cols=wdg.SelectMultiple(
    options=["M_0_to_4", "M_5_to_9", "M_10_to_14", "M_15_to_19", "M_20_to_24", "M_25_to_29", "M_30_to_34", "M_35_to_39", "M_40_to_44", "M_45_to_49", "M_50_to_54", "M_55_to_59", "M_60_to_64", "M_65_to_69", "M_70_to_74", "M_75_to_79", "M_80_to_84", "M_85_to_89", "M_90_to_94", "M_90+"],
    value=[],
    rows= 20, # rows of the selection box
    description='Male Age Bands',
    disabled=False
)

timeslider3=wdg.IntSlider(
    value=9,
    min=1,
    max=12,
    step=1,
    description='Monthly Intervall',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

controls = wdg.HBox([female_age_cols, male_age_cols, timeslider3])

def death_gender_graph(femagecols, menagecols, timevalue):

    # our callback function.
    ncols=len(femagecols)
    m_value = str(timeslider3.value) + "m"


    if ncols>0:
        death_df_mm= death_df.resample(m_value).sum()
        death_df_m=death_df.resample("1m").sum()        # explain this in the header -> we are fixing this at a one month intervall to have the right resolution for deaths
        death_df_mm.plot(y=list(femagecols + menagecols), kind="area", logy = False, use_index=True, figsize=(20,10)) # graphcolumns is a tuple - we need a list
        ax = death_df_m["DailyDeaths"].plot(secondary_y=True, color='k')    # check here if you want daily or resampled data
        ax.set_ylabel('Daily Deaths')

    else:
        
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    
  
graph_death_gender=wdg.interactive_output(death_gender_graph, {'femagecols': female_age_cols, "menagecols" : male_age_cols, "timevalue": timeslider3}) # clean up slider names + order

display(controls, graph_death_gender)

Output()

In [58]:
### refreshing the charts ###

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """

    # trying a loop here
    chartcontr_lst = [pcr_cols, vaxcols,] # loop over one control widget for eacg graph

    for chart in chartcontr_lst:    # go through the steps of changing / unchanging an option

        current=pcr_cols.value
        if current==chart.options[0]:
            other=chart.options[1]
        else:
            other=chart.options[0]
        chart.value=other # forces the redraw
        chart.value=current # changing it back

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge the data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*